# Загрузка пакетов, библиотек, моделей
Не забудьте сделать "restart runtime" в Colab

In [1]:
%%capture
!pip install natasha
!pip install pymorphy2

In [2]:
# Не забудьте сделать "restart runtime" 
!pip install deeppavlov
!python -m deeppavlov install ner_rus_bert
!pip install pandas==1.3.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pandas-0.25.3-cp37-cp37m-manylinux1_x86_64.whl (10.4 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.20.2 requires pandas>=1.1, but you have pandas 0.25.3 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.18.0 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.4.1 which is incompatible.
prophet 1.1 requires pandas>=1.0.4, but you have pandas 0.25.3 which is incompatible.
plotnine 0.8.0 requires numpy>=1.19.0, but you have numpy 1.18.0 which is incompatible.
plotnine 0.8.0 requires pandas>=1.1.0, but you have pandas

In [3]:
# Не забудьте сделать "restart runtime" 

In [4]:
import pandas as pd

df = pd.read_csv('test_data.csv')
df.head(3)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день


In [5]:
from yargy import Parser, rule, and_, not_
from yargy.predicates import gram
from yargy.pipelines import morph_pipeline
import pymorphy2
from yargy import Parser
from yargy.pipelines import morph_pipeline

In [6]:
from deeppavlov import configs, build_model

ner = build_model(configs.ner.ner_rus_bert, download=True)

2022-08-26 11:52:22.926 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes
2022-08-26 11:52:27.574 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]  

# Парсеры по правилам

In [7]:
# https://sinonim.org/
# Dictionary greeting
rule_greeting = morph_pipeline(['здравствуйте', 
                                'добрый день', 
                                'приветствую', 
                                'доброе утро', 
                                'добрый вечер',
                                'привет',
                                ])
parser_greeting = Parser(rule_greeting)

In [8]:
# Dictionary parting
rule_parting = morph_pipeline(['до свидания',
                               'до встречи',
                               'пока',
                               'будьте здоровы',
                               'до вечера',
                               'всех благ',
                               'до завтра',
                               'всего доброго',
                                ])
parser_parting = Parser(rule_parting)

In [9]:
# Dictionary company
rule_company = morph_pipeline(['компании',
                               'организация',
                               ])
parser_company = Parser(rule_company)

In [10]:
#Manager name lookup rule
PRONOUN = morph_pipeline(['меня',
                          'моё',   
                          'это',
                          ])
CALLED = morph_pipeline(['зовут',
                         'имя',
                         'говорит',
                         ])
NAME = and_(gram('Name'),
            not_(gram('Abbr')),
            )
rule_name_manager = rule(PRONOUN,
                         CALLED.optional(),
                         NAME,
                         )
parser_name_manager = Parser(rule_name_manager)

# Служебные функции

In [11]:
def preprocessing_text(text: str, pos: int, lenght_company : int=3) -> str:
    """
    The function capitalizes multiple words (lenght_company) in the text (text)
    after the specified position (pos)
    """
    index_list :list = []
    for indx, s in enumerate(text[pos:]):
        if s == ' ':
            index_list.append(pos + indx + 1)
    string = text[:pos]
    for indx, s in enumerate(text[pos:]):
        if (pos + indx) in index_list[:lenght_company]:
            string += s.upper()
        else:
            string += s
    return string

In [12]:
def _print_NER(position_ner: list, tok_list: list) -> list:
    result :list = []
    for i in position_ner:
        result.append(' '.join(tok_list[i[0]:i[1]]))
    return result

def find_NER(tokens: list, tags: list, ner: str='ORG') -> list:   
    """
    The function returns the NER in the text found by the model deeppavlov.
    ner: 'ORG', 'LOC', 'PER' - The named entity you are looking for

    """ 
    tok_list :list = []
    tag_list :list = []
    for tok, tag in zip(tokens[0], tags[0]):
        if ner in tag:
            tok_list.append(tok)
            tag_list.append(tag)

    position_ner :list = []
    if tag_list:
        index_list :list = []
        for indx, el in enumerate(tag_list):
            if el[0] == 'B':
                index_list.append(indx)

        for indx, i in enumerate(index_list[:-1]):
            position_ner.append((i, index_list[indx+1]))
        position_ner.append((index_list[-1], len(tag_list)))
    return _print_NER(position_ner, tok_list)

# Основной код

In [13]:
df_output = df.copy()
cnt :int = 0  # Dialogue counter
morph = pymorphy2.MorphAnalyzer()

greeting :list = []
parting :list = []
mandatory_requirement :int = 0

print(f'Dialogue №{cnt}')
for row in df.itertuples(index=True):

    if row.role == 'manager':
        for match in parser_greeting.findall(row.text):
            greeting = [_.value for _ in match.tokens]
            if greeting:
                print(f'Greeting string: line_n={row.line_n}, text={row.text}')
                df_output.loc[row.Index, 'greeting'] = 1
                mandatory_requirement += 1
                break

        for match in parser_parting.findall(row.text):
            parting = [_.value for _ in match.tokens]
            if parting:
                print(f'Parting string: line_n={row.line_n}, text={row.text}')
                df_output.loc[row.Index, 'parting'] = 1

                mandatory_requirement += 1
                break

        for match in parser_name_manager.findall(row.text):
            print(f'Introduced himself string: line_n={row.line_n}, text={row.text}')
            print(f'Manager\'s name: {match.tokens[-1].value.capitalize()}')

        for match in parser_company.findall(row.text):
            string = preprocessing_text(text=row.text, pos=list(match.span)[0])
            tokens, tags = ner([string])
            company = find_NER(tokens, tags, ner='ORG')
            if company:
                print(f'Company\'s name: {company}')

    if cnt != int(row.dlg_id):
        check = True if mandatory_requirement == 2 else False
        print(f'Greeting and parting: {check}')
        mandatory_requirement = 0    
        cnt += 1
        print(f'\nDialogue №{cnt}')

check = True if mandatory_requirement == 2 else False
print(f'Greeting and parting: {check}')

Dialogue №0
Greeting string: line_n=1, text=Алло здравствуйте
Introduced himself string: line_n=3, text=Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
Manager's name: Ангелина
Company's name: ['Диджитал Бизнес']
Parting string: line_n=108, text=Всего хорошего до свидания
Greeting and parting: True

Dialogue №1
Greeting string: line_n=1, text=Алло здравствуйте
Introduced himself string: line_n=2, text=Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается
Manager's name: Ангелина
Company's name: ['Диджитал Бизнес']
Parting string: line_n=54, text=До свидания
Greeting and parting: True

Dialogue №2
Greeting string: line_n=2, text=Алло здравствуйте
Introduced himself string: line_n=3, text=Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там
Manager's name:

In [14]:
# Adding information to the 'greeting' and 'parting' columns of the output dataframe
df_output.fillna(0, inplace=True)
df_output[['greeting', 'parting']] = df_output[['greeting', 'parting']].astype('int')
df_output.to_csv('test_data_output.csv', index=False)